# Data Science Discovery Project - UCSC MARINe Academic Research

## Import Modules and Datasets

In [260]:
%matplotlib notebook
import pandas as pd
import folium
from folium import plugins
from folium.plugins import Search
import json
from matplotlib import cbook
from matplotlib import cm
from matplotlib.colors import LightSource
import matplotlib.pyplot as plt
import numpy as np

In [261]:
# === import dataframe
df = pd.read_csv("homepage_sites_data.csv")
df.head(10)

,intertidal_sitename,site_latitude,site_longitude,linked_site
0,Bob Creek,44.244640,-124.11430,bob_creek.html
1,Bodega,38.318194,-123.07365,bodega.html
2,Buck Gully South,33.588246,-117.86736,buck_gulley_south.html
3,Cape Arago,43.308915,-124.40053,cape_arge.html
4,Fogarty Creek,44.837085,-124.05778,fogarty_creek.html
5,Heisler Park,33.542594,-117.78928,heisler_park.html
6,Point Fermin,33.706790,-118.28614,point_fermin.html
7,White Point,33.715730,-118.31999,white_point.html


## Create an interactive map

In [262]:
# =================== compute middle point =================== 
middle_coordinate = [df["site_latitude"].mean(), df["site_longitude"].mean()]

# create a map
map = folium.Map(location=[middle_coordinate[0], middle_coordinate[1]], 
                 zoom_start=5)


for i in range(0, len(df)):
    map.add_child(
            folium.Marker(
                location = [df.iloc[i]["site_latitude"],df.iloc[i]["site_longitude"]],
                popup = folium.Popup(html="<strong>Sitename</strong>: " + str(df.iloc[i]["intertidal_sitename"]) \
                                            +"<br>" \
                                            +"<strong>Latitude</strong>: " + str(df.iloc[i]["site_latitude"]) \
                                            +"<br>" \
                                            +"<strong>Longitude</strong>: " + str(df.iloc[i]["site_longitude"]) \
                                            +"<br>" \
                                            +"Click <a href = "+df.iloc[i]["linked_site"]+">here</a> to see detailed information", 
                                            max_width=500),
                    icon=folium.Icon(color="blue",icon="info-sign")))
# show lat and lon
map.add_child(folium.LatLngPopup())
map.add_child(plugins.MeasureControl(position ='topleft'))



#  =================== Create title  =================== 
loc = 'UCSC MARINe Site Statistics'
title_html = '''
             <h1 align="center" style="font-size:30px; color:#5F9EA0"><b>{}</b></h1>
             '''.format(loc)

map.get_root().html.add_child(folium.Element(title_html))


# =================== create search box =================== 
useful_cols = ['intertidal_sitename', 'site_latitude', 'site_longitude']
df_geo = df[useful_cols]

# Convert to json data
def df_to_geojson(df_json, properties, lat='site_latitude', lng='site_longitude'):
    geojson = {'type': 'FeatureCollection', 'features': []}

    # 遍历dataframe的每一行，并转换为json格式
    for _, row in df_json.iterrows():
        # create a feature template to fill in
        feature = {'type': 'Feature',
                   'properties': {},
                   'geometry': {'type': 'Point', 'coordinates': []}}

        # 填写坐标
        feature['geometry']['coordinates'] = [row[lng], row[lat]]

        # for each column, get the value and add it as a new feature property
        for prop in properties:
            feature['properties'][prop] = row[prop]

        # add this feature (aka, converted dataframe row) to the list of features inside our dict
        geojson['features'].append(feature)

    return geojson

# create layers
geojson_dict = df_to_geojson(df_geo, properties=useful_cols)
geojson_str = json.dumps(geojson_dict, indent=2)
geojson_obj = folium.GeoJson(geojson_dict,
                             marker = folium.CircleMarker(radius = 3, # Radius in metres
                                                          weight = 0, #outline weight
                                                          fill_color = '#000000',
                                                          fill_opacity = 1)
                             ,show=False).add_to(map)

# show search boxes
factorysearch = Search(layer=geojson_obj,
                       geom_type='Point',
                       placeholder="Please input the site name",
                       collapsed=False,
                       search_label='intertidal_sitename',
                       search_zoom=6,
                       position='topright'
                       ).add_to(map)

## Save the website to local disk

In [263]:
map.save('UCSC_MARINe_Homepage.html')